# Sismique active S : Conversion de formats

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from scipy import stats

## Lecture de la géométrie du profil

In [2]:
from geometry import *

In [3]:
!cat geometry.py

import numpy as np
import pandas as pd
import os

gnss = pd.read_csv('MUCKENBACH_OS_2024.csv')
gnss['geophone'] = gnss['Name'].map(lambda e: int(e.split('.')[-1]))
gnss = gnss.set_index('geophone')
gnss = gnss.sort_index()

files_pointage = [f for f in os.listdir('data/pointage') if f.endswith('txt')]

dx = 4
offset = 2
ng = len(gnss['A'].unique())
ns = len(files_pointage)
x_geophs = np.arange(ng)*dx # géophones espacés de 4m
x_shots = np.arange(ns)*dx + offset # sources espacées de 4m entre 2 géophones
x = np.concatenate((x_geophs, x_shots))
x.sort()
y_geophs = gnss['A'].values
y = np.interp(x, x_geophs, y_geophs) # interpolation de l'altitude des capteurs (mesurées) pour estimer celle des sources
y_shots = y[0:-1:2]

In [4]:
gnss.head()

,Name,Latitude,Longitude,EllipsoidHeight,TimeStamp,X,Y,A
geophone,,,,,,,,
1,ONDES.GR1.1,48.491419,7.288737,713.242974,2024-09-04T15:10:04.420,6.829844e+06,1.016730e+06,664.504873
2,ONDES.GR1.2,48.491391,7.288704,713.303219,2024-09-04T15:10:28.280,6.829841e+06,1.016728e+06,664.564993
3,ONDES.GR1.3,48.491363,7.288673,713.271782,2024-09-04T15:10:46.900,6.829837e+06,1.016726e+06,664.533426
4,ONDES.GR1.4,48.491334,7.288640,713.350197,2024-09-04T15:11:05.850,6.829834e+06,1.016723e+06,664.611712
5,ONDES.GR1.5,48.491305,7.288607,713.588551,2024-09-04T15:11:25.210,6.829831e+06,1.016721e+06,664.849936


In [5]:
print('Nombre de geophones', ng)
print('Nombre de shots (sources)', ns)
print('Nombre total de données', ns*ng)

Nombre de geophones 21
Nombre de shots (sources) 20
Nombre total de données 420


## Pygimli
data.sgt

In [6]:
path = 'data/sgt'

In [7]:
xy = np.array((x, y)).T # positions de tous les capteurs et receveurs

In [8]:
s = np.zeros(ns*ng, dtype=int)
g = np.zeros(ns*ng, dtype=int)
t = np.zeros(ns*ng, dtype=float)
i = 0
with open('data/pointage.txt', 'r') as f: # reading this terrible file format TODO : read from individual files
    for line in f.readlines():
        l = line.split()
        if len(l)>2:
            ti = l[0]
            d = l[1]
            f = open(f'data/pointage/{ti}.txt', 'w') # clear content of old file and opens it
        else:
            if l:
                d = l[0]
            else:
                continue
        shot = int(float(ti)/1000-2)//4+1
        geophone = int(float(l[-1]))
        
        if shot == geophone:
            pass
            #print("Retire le pointé", geophone, "du shot", shot, "(source au même endroit que le pointé)")
            #continue
        # le shot et le geophone sont au même endroit donc on peut utiliser une position pour les 2
        
        s[i] = 2*shot
        g[i] = 2*geophone-1
        t[i] = float(d)
        i += 1
        f.write(f'{d}\n') # f'{d} {l[-1]}\n'
f.close()

In [9]:
print('Nombre de geophones', ng)
print('Nombre de shots (sources)', ns)
print('Nombre total de positions', len(xy)) # car les shots sont aux positions des capteurs donc on ne les compte pas
print('Nombre total de données', ns*ng)
print('Nombre total de données réelles', len(t)) # en réalité on en retire 1 par shot soit 20

Nombre de geophones 21
Nombre de shots (sources) 20
Nombre total de positions 41
Nombre total de données 420
Nombre total de données réelles 420


In [10]:
np.savetxt(f'{path}/positions.txt', xy)
np.savetxt(f'{path}/shots.txt', s, fmt='%d')
np.savetxt(f'{path}/geophones.txt', g, fmt='%d')
np.savetxt(f'{path}/times.txt', t)
np.savetxt(f'{path}/sgt.txt', np.array([s, g, t]).T) # , delimiter='\t', fmt='%.5f'

In [11]:
with open(f'{path}/data.sgt', 'w') as f:
    f.write(f'{len(xy)} # shot/geophone points\n')
    f.write('#x	y\n')
    #f.write(open('xy', 'r').read())
    for pos in xy:
        x, y = pos
        f.write(f'{x} {y}\n')
    f.write(f'{len(t)} # measurements\n')
    f.write('#s	g	t\n')
    #f.write(open('sgt', 'r').read())
    for i in range(len(t)):
        f.write(f'{s[i]} {g[i]} {t[i]}\n')

## Rayinvr

[Rayinvr](https://github.com/hzhu212/rayinvr)

In [12]:
path = 'data/rayinvrmodel'

In [13]:
# tx.in
txin = []
dmin = 2 # distance min from source (all bellow removed)
for i, x_shot in enumerate(x_shots):
    shot_data = []
    file = f"data/pointage/{x_shot:03d}000.txt"
    ts = np.loadtxt(file)*1000
    for j, x_geoph in enumerate(x_geophs):
        if x_shot - dmin <= x_geoph <= x_shot + dmin:
            #continue
            pass
        nc = 2
        shot_data.append([x_geoph, ts[j], 1.5, nc])
    shot_data.insert(i, [x_shot, 1, 0, 0])
    if shot_data[0][0] < x_shot:
        shot_data.insert(0, [x_shot, -1, 0, 0])
    txin += shot_data
txin.append([0, 0, -1, -1])

np.savetxt(f'{path}/tx.in', txin, fmt="% 10.3f% 10.3f% 10.3f% 10d")

In [14]:
def create_velocity_model(k=0, x_geophs=x_geophs, parameter=20, vary=True, y=None, x_borders=5, topo=0, data='', ncolmax=10):
    """
    create the v.ori velocity model for rayinvr
    k : number of the layer, 0 for 1
    x_geophs : pos of geophones (1D x distance)
    parameter : parameter of the layer, depth or speed, either value either array of length x_geoph+2
    vary : bool, should the parameter evolves thru iterations
    y : topography of the layer of length x_geoph
    x_borders : what is added to each sides
    topo : topography of size x_geoph+2
    data : string to write on
    ncolmax : number of columns to write on
    """
    if y is not None:
        topo = np.concatenate([[y[0]], y, [y[-1]]])
    x_vs = np.concatenate([[x_geophs[0]-x_borders], x_geophs, [x_geophs[-1]+x_borders]])
    z_vs = np.zeros_like(x_vs) + parameter + topo
    c_vs = np.zeros_like(x_vs, dtype=int) + vary
    v_ori = np.array([x_vs, z_vs, c_vs])
    n_cols = len(x_vs)//10+1
    for i in range(n_cols):
        for j in range(3):
            if j<2:
                data += f'{int(i!=n_cols-1 or j==0)+int(j==0)*k: 2d} '+''.join(f'{e: 7.2f}' for e in v_ori[j, i*10:(i+1)*10])
            else:
                data += 3*' '+''.join(f'{int(e): 7d}' for e in v_ori[j, i*10:(i+1)*10])
            data += '\n'
    return data

In [15]:
# v.ori
# Topo to relative depth
y = y_geophs
y = np.max(y) - np.min(y) - (y - np.min(y))
# First layer
k = 0
vori = create_velocity_model(k=k, x_geophs=x_geophs, parameter=0, vary=False, y=y) # Layer depth
vori = create_velocity_model(k=k, x_geophs=x_geophs, parameter=0.24, vary=False, data=vori) # Layer upper speed
vori = create_velocity_model(k=k, x_geophs=x_geophs, parameter=0.60, vary=True, data=vori) # Layer upper speed
# Second layer
k = 1
vori = create_velocity_model(k=k, x_geophs=x_geophs, parameter=20, vary=True, data=vori, y=y) # Layer depth
vori = create_velocity_model(k=k, x_geophs=x_geophs, parameter=1.30, vary=False, data=vori) # Layer upper speed
vori = create_velocity_model(k=k, x_geophs=x_geophs, parameter=2.75, vary=False, data=vori) # Layer upper speed
# Last layer
vori += ' 3   -5.00  85.00\n'
vori += ' 0   40.00  40.00\n'
# Save
with open(f'{path}/v.ori', 'w') as f:
    f.write(vori)

In [16]:
# parameters
print(f"ishot={'2,'*ns}")
print(f"xshot={','.join(map(str, map(float, x_shots)))},")

ishot=2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
xshot=2.0,6.0,10.0,14.0,18.0,22.0,26.0,30.0,34.0,38.0,42.0,46.0,50.0,54.0,58.0,62.0,66.0,70.0,74.0,78.0,


In [17]:
# v.in
!cp -f data/rayinvrmodel/v.ori data/rayinvrmodel/v.in